In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install pyedflib
!pip install git+https://github.com/EtienneCmb/visbrain
!pip install beautifultable

In [6]:
import visbrain as vb
#from visbrain.io import write_fig_hyp, read_hypno
import pandas as pd
import pyedflib as edf
import matplotlib.pyplot as plt 
import glob
import numpy as np
import os
import keras
import keras.backend as K
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten
import tensorflow as tf
from IPython.display import display
import warnings
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras import losses
from keras import losses
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from keras import optimizers, losses, activations, models
import time
import matplotlib
import beautifultable
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, f1_score
from beautifultable import BeautifulTable
from keras.utils.np_utils import to_categorical
from keras.layers import BatchNormalization

warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
def retrieveHypnogramData(path_to_hypno, path_to_datafile):
    grid = True     # display the grid
    ascolor = True  # plt as color or in black and white
    file = None     # Name of the file to be saved example : 'myfile.png'
    data, sf = read_hypno(path_to_hypno,datafile=path_to_datafile)
    print(len(data))
    print(sf)
    write_fig_hyp(data, 1, grid=grid, ascolor=ascolor, file=file)
    return data,sf

def retrieveHypnogramData_2v(path_to_hypno, path_to_datafile):
    data, sf = read_hypno(path_to_hypno,datafile=path_to_datafile)
    return data,sf

def seperateSignalsFromEDF(datafile):
    d = edf.EdfReader(datafile)
    n = 2
    eegSignals = pd.DataFrame(columns=[['Signal']],index=range(n))
    if n != 0:
        for i in np.arange(n):
            sigbuf= d.readSignal(i)
            print(d.getSignalLabels())
            eegSignals.loc[i].Signal=[sigbuf]
    d._close()
    del d
    return eegSignals

def divideSignalInSleepStages(sleep_stages,sleep_signal):
    chunks = len(sleep_signal)//len(sleep_stages)
    step = 0
    nextStep = int(step+chunks)
    data_set = pd.DataFrame(columns=[['Class','Signal']],index=range(len(sleep_stages)-1))
    for i in range(len(sleep_stages)-1):
        subsignal = sleep_signal[step:nextStep]
        step = nextStep
        nextStep=step+chunks
        signal_with_stage = np.array(subsignal)
        data_set.loc[i].Class = int(sleep_stages[i])
        data_set.loc[i].Signal=[signal_with_stage]
    return data_set

def plot_Signal(eegSignals, n):
    signal = get_signal(eegSignals,n)
    #print(signal.shape)
    #plt.plot(signal)
    #plt.xlabel('Time [samples]')
    #plt.ylabel('Amplitude')
    #plt.grid(True)
    #plt.show()
    
def get_signal(data_set,n):
    eegSignal = data_set.iloc[n]['Signal']
    signal = eegSignal[0]
    return signal
  
def getChannelData(file_list, data):
    eegSignals = []
    for datafile in file_list:
        eegSignals = seperateSignalsFromEDF(datafile)
    plot_Signal(eegSignals,1)
    #devide data into classes
    signal_data1 = pd.DataFrame(columns=[['Class','Signal']])
    signal_data2 = pd.DataFrame(columns=[['Class','Signal']])
    for i in range(2):
        sigbuf = get_signal(eegSignals,i)
        data_set = divideSignalInSleepStages(data, sigbuf)
        if i == 0:
            signal_data1 = data_set
        else:
            signal_data2 = data_set
    return signal_data1,signal_data2
  
def keras_auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def data_generation2(signal_data1, signal_data2, batch_size, expand_dim=True,dim_shape = (1,500,2)):
    # Initialization
    #X = np.empty((self.dim))
    y = np.empty((batch_size), dtype=int)
    #s = self.dataSet.iloc[0]['Signal']
    chunk = dim_shape[1]
    parts = int(3000 / dim_shape[1])
    a = np.empty([batch_size, dim_shape[2], dim_shape[1]])
    # Generate data
    k=0
    for i in range(len(signal_data2)-1):
        s1 = signal_data1.iloc[i]['Signal']
        s2 = signal_data2.iloc[i]['Signal']
        cl = signal_data1.iloc[i]['Class']
        stepOne = 0
        sign = s1[0]
        sign2 = s2[0]
        clas1 = cl[0]
        for j in range(parts):
            a[k, :] = np.array([sign[stepOne:(stepOne + chunk)], sign2[stepOne:(stepOne + chunk)]])
            stepOne = stepOne + chunk
            y[k] = clas1
            k = k+1
    w = a
    a_new = np.reshape(w, (w.shape[0], w.shape[2], 2))
    #output shape(32, 1, 500, 2)
    if(expand_dim):
      a_new = np.expand_dims(a_new, axis=1)
    X = a_new
    print(X.shape)
    return X, y
  
def data_generation_sleep(signal_data1, signal_data2, batch_size, expand_dim=True,dim_shape = (1,500,2)):
    # Initialization
    #X = np.empty((self.dim))
    y = np.empty((batch_size), dtype=int)
    #s = self.dataSet.iloc[0]['Signal']
    chunk = dim_shape[1]
    parts = int(3000 / dim_shape[1])
    a = np.empty([batch_size, dim_shape[2], dim_shape[1]])
    # Generate data
    k=0
    nomoda  = 0
    for i in range(len(signal_data2)-1):
        s1 = signal_data1.iloc[i]['Signal']
        s2 = signal_data2.iloc[i]['Signal']
        cl = signal_data1.iloc[i]['Class']
        stepOne = 0
        sign = s1[0]
        sign2 = s2[0]
        clas1 = cl[0]
        
        if (clas1 >= 1) and (clas1 <= 4):
          clas1 = 1
          
        if (clas1<=0) or (clas1 > 4):
          clas1 = 0

        for j in range(parts):
          a[k, :] = np.array([sign[stepOne:(stepOne + chunk)], sign2[stepOne:(stepOne + chunk)]])
          stepOne = stepOne + chunk
          y[k] = clas1
          k = k+1
            #print(clas1)
    w = a
    a_new = np.reshape(w, (w.shape[0], w.shape[2], 2))
    #output shape(32, 1, 500, 2)
    if(expand_dim):
      a_new = np.expand_dims(a_new, axis=1)
    X = a_new
    print(X.shape)
    #print(y)
    
    y[y>4]=0
    y[y<0]=0
  
    return X,keras.utils.to_categorical(y)
  
def prepare_data_sleep(expand_dim=True,data_shape = (1,500,2)):
    signal_data1 = pd.DataFrame(columns=[['Class', 'Signal']])
    signal_data2 = pd.DataFrame(columns=[['Class', 'Signal']])

    hypnogram_files = [x for x in os.listdir('/content/gdrive/My Drive/EEG/Test') if
                       x.endswith("gram.edf")]
    eeg_files = [x for x in os.listdir('/content/gdrive/My Drive/EEG/Test') if
                 x.endswith("PSG.edf")]
    hypnogram_files.sort()
    eeg_files.sort()
    for i in range(1):
        hip_fil = '/content/gdrive/My Drive/EEG/Test/SC4001EC-Hypnogram.edf' 
        eeg_fil = '/content/gdrive/My Drive/EEG/Test/SC4001E0-PSG.edf'
        data, sf = retrieveHypnogramData_2v(hip_fil, eeg_fil)
        sig_data1, sig_data2 = getChannelData([eeg_fil], data)
        frames = [signal_data1, sig_data1]
        signal_data1 = pd.concat(frames)
        frames2 = [signal_data2, sig_data2]
        signal_data2 = pd.concat(frames2)
    X,y = data_generation_sleep(signal_data1, signal_data2, len(signal_data2)*6,expand_dim, dim_shape = data_shape)
 
    return X,y  


def prepare_data(expand_dim=True,data_shape = (1,500,2)):
    signal_data1 = pd.DataFrame(columns=[['Class', 'Signal']])
    signal_data2 = pd.DataFrame(columns=[['Class', 'Signal']])

    hypnogram_files = [x for x in os.listdir('/content/gdrive/My Drive/EEG/Test') if
                       x.endswith("gram.edf")]
    eeg_files = [x for x in os.listdir('/content/gdrive/My Drive/EEG/Test') if
                 x.endswith("PSG.edf")]
    hypnogram_files.sort()
    eeg_files.sort()
    for i in range(1):
        hip_fil = '/content/gdrive/My Drive/EEG/Test/SC4001EC-Hypnogram.edf' 
        eeg_fil = '/content/gdrive/My Drive/EEG/Test/SC4001E0-PSG.edf'
        data, sf = retrieveHypnogramData_2v(hip_fil, eeg_fil)
        sig_data1, sig_data2 = getChannelData([eeg_fil], data)
        frames = [signal_data1, sig_data1]
        signal_data1 = pd.concat(frames)
        frames2 = [signal_data2, sig_data2]
        signal_data2 = pd.concat(frames2)
    X,y = data_generation2(signal_data1, signal_data2, len(signal_data2)*6,expand_dim, dim_shape = data_shape)
    return X,y
    
def plot_hypnogram(stages, labels=None, title='', ax1=None, **kwargs):
    if labels is None:
        if np.max(stages)==4:
            print('assuming 0=W, 1=S1, 2=S2, 3=SWS, 4=REM')
            labels = ['W', 'S1', 'S2', 'SWS', 'REM']
        if np.max(stages)==5:
            print('assuming 0=W, 1=S1, 2=S2, 3=S3, 4=S4, 5=SWS')
            labels = ['W', 'S1', 'S2', 'S3', 'S4', 'REM']
        if np.max(stages)==8:
            print('assuming 0=W, 1=S1, 2=S2, 3=S3, 4=S4, 5=SWS')
            labels = ['W', 'S1', 'S2', 'S3', 'S4', 'REM', 'Movement']
    labels_dict = dict(zip(np.arange(len(labels)),labels))

    x = []
    y = []
    for i in np.arange(len(stages)):
        s = stages[i]
        if labels_dict[s]=='W':   p = -0
        if labels_dict[s]=='REM': p = -1
        if labels_dict[s]=='S1':  p = -2
        if labels_dict[s]=='S2':  p = -3
        if labels_dict[s]=='SWS': p = -4
        if labels_dict[s]=='S3': p = -4
        if labels_dict[s]=='S4': p = -5
        if i!=0:
            y.append(p)
            x.append(i-1)   
        y.append(p)
        x.append(i)
        
    x = np.array(x)*30
    y = np.array(y)
    if ax1 is None:
        fig = plt.figure(figsize=[8,2])
        ax1 = fig.add_subplot(111)
    formatter = matplotlib.ticker.FuncFormatter(lambda s, x: time.strftime('%H:%M', time.gmtime(s)))
    ax1.xaxis.set_major_formatter(formatter)
    ax1.plot(x,y, **kwargs)
    plt.yticks([0,-1,-2,-3,-4,-5], ['Nomoda','REM', 'N1', 'N2', 'N3' ])
    plt.xticks(np.arange(0,x[-1],3600))#3600
    plt.xlabel('Laiks')
    plt.ylabel('Miega stadijas')
    plt.title(title)
    plt.tight_layout()

def my_predict(model, data, TL, classes = True):
    wake = 0; s1 = 1; s2 = 2; sws = 3; rem = 4
    mapping = {0:wake, 1:s1, 2:s2, 3:sws, 4:rem}
    cnn_preds = model.predict(data)
    scores = model.evaluate(data, TL, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    preds = np.roll(cnn_preds,-4, axis=0)
    if classes:
        preds = np.argmax(preds,1)
        preds = [mapping[x] for x in preds]
    return preds
  
def my_predict_sleep(model, data, TL, classes = True):
    wake = 0; s1 = 1;
    mapping = {0:wake, 1:s1}
    cnn_preds = model.predict(data)
    scores = model.evaluate(data, TL, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    preds = np.roll(cnn_preds,-2, axis=0)
    #print(preds)
    if classes:
        preds = np.argmax(preds,1)
        preds = [mapping[x] for x in preds]
    return preds
  
def show_sample_hypnogram(filename, start=None,stop=None, title='True Sleep Stage'):
    hypno = np.loadtxt(filename)
    hypno = hypno[start:stop]
    print(hypno)
    plot_hypnogram(hypno,title = title)
    
def getStageTrainingAndValidationData(signal_data1,signal_data2,classOfSamples,traingSize,testSize,validationSize):
    eeg_test = pd.DataFrame(columns=[['Class', 'Signal']])
    eeg_val = pd.DataFrame(columns=[['Class', 'Signal']])
    eeg_train = pd.DataFrame(columns=[['Class', 'Signal']])
    temp = pd.DataFrame(columns=[['Class', 'Signal']], index=range(1))
    #k = 0
    for i in range(len(signal_data1)- 1):
        clas = signal_data1.iloc[i]['Class']
        sig = signal_data1.iloc[i]['Signal']
        sig2 = signal_data2.iloc[i]['Signal']
        temp.loc[0].Class = clas[0]
        temp.loc[0].Signal = [[sig[0], sig2[0]]]
        if clas[0] == classOfSamples:
            #k = 1+k
            if len(eeg_test)<testSize:
                frames = [eeg_test, temp]
                eeg_test = pd.concat(frames)
            elif len(eeg_val)<validationSize:
                frame = [eeg_val, temp]
                eeg_val = pd.concat(frame)
            elif len(eeg_train)<traingSize:
                frames = [eeg_train, temp]
                eeg_train = pd.concat(frames)
    #print(k)
    return eeg_train,eeg_test,eeg_val
  
def getSleepStageTrainingAndValidationData(signal_data1,signal_data2,traingSize,testSize,validationSize):
    eeg_test = pd.DataFrame(columns=[['Class', 'Signal']])
    eeg_val = pd.DataFrame(columns=[['Class', 'Signal']])
    eeg_train = pd.DataFrame(columns=[['Class', 'Signal']])
    temp = pd.DataFrame(columns=[['Class', 'Signal']], index=range(1))
    #k = 0
    for i in range(len(signal_data1)- 1):
        clas = signal_data1.iloc[i]['Class']
        sig = signal_data1.iloc[i]['Signal']
        sig2 = signal_data2.iloc[i]['Signal']
        temp.loc[0].Class = clas[0]
        temp.loc[0].Signal = [[sig[0], sig2[0]]]
        if clas[0] != 0:
            temp.loc[0].Class = 1
            if len(eeg_test)<testSize:
                frames = [eeg_test, temp]
                eeg_test = pd.concat(frames)
            elif len(eeg_val)<validationSize:
                frame = [eeg_val, temp]
                eeg_val = pd.concat(frame)
            elif len(eeg_train)<traingSize:
                frames = [eeg_train, temp]
                eeg_train = pd.concat(frames)
        
    return eeg_train,eeg_test,eeg_val  
  
def predict_eeg_signal_sleep(model,filename,clases,expand_dim=True, dat_shape = (1,500,2)):
  data2,y_labels = prepare_data_sleep(expand_dim, data_shape =dat_shape )
  #y_labels[y_labels>4]=0
  #y_labels[y_labels<0]=0
  
  #print(y_labels.size)
  #TL2 = keras.utils.to_categorical(y_labels, num_classes=clases)
  #filename = 'sample_test_hypnogram_CNN'
  hypnograms = True
  print('Predicting...')
  preds1 = my_predict_sleep(model, data2,y_labels,classes=True)
  #np.savetxt(filename + '.csv', preds1, fmt='%d')
  #print('Predictions saved to {}'.format (filename + '.csv'))
  #if hypnograms:
  #    plot_hypnogram(preds1, title ='Predictions for {}'.format(os.path.basename(filename)))
  #    plt.savefig(filename + '.hyp.png')
  #    print('Hynograms saved to {}'.format(filename + '.hyp.png'))
  return y_labels, preds1  
  
def predict_eeg_signal(model,filename,clases,expand_dim=True, dat_shape = (1,500,2)):
  data2,y_labels = prepare_data(expand_dim, data_shape =dat_shape )
  y_labels[y_labels>4]=0
  y_labels[y_labels<0]=0
  
  
  print(y_labels)
  TL2 = keras.utils.to_categorical(y_labels, num_classes=clases)
  #filename = 'sample_test_hypnogram_CNN'
  hypnograms = True
  print('Predicting...')
  preds1 = my_predict(model, data2,TL2,classes=True)
  np.savetxt(filename + '.csv', preds1, fmt='%d')
  print('Predictions saved to {}'.format (filename + '.csv'))
  if hypnograms:
      plot_hypnogram(preds1, title ='Predictions for {}'.format(os.path.basename(filename)))
      plt.savefig(filename + '.hyp.png')
      print('Hynograms saved to {}'.format(filename + '.hyp.png'))
  return y_labels, preds1

def plot_original_eeg(filename,expand_dim=True, dat_shape = (1,500,2)):
  hypnograms = True
  data2,y_labels = prepare_data(data_shape =dat_shape)
  y_labels[y_labels>4]=0
  np.savetxt(filename + '.csv', y_labels, fmt='%d')
  print('Predictions saved to {}'.format (filename + '.csv'))
  if hypnograms:
      plot_hypnogram(y_labels, title ='Original signal for {}'.format(os.path.basename(filename)))
      plt.savefig(filename + '.hyp.png')
      print('Hynograms saved to {}'.format(filename + '.hyp.png'))
      
def train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = False,expand = True, verbose = True,dim_shape = (64,500,2)):
  if(weight_loaded):
    model.load_weights(weights_name)

  list_of_indexes = np.arange(0,dataSet.shape[0],1)
  labels = dataSet['Class'].values.tolist()
  list_of_val_index = np.arange(0,validDataSet.shape[0],1)
  labels_val = validDataSet['Class'].values.tolist()

  tra_generator = MyDatGen(list_of_Indexes=list_of_indexes,batch_size=batch_size ,dataSet=dataSet,labels=labels, expand_dim = expand, dim =dim_shape)
  val_generator = MyDatGen(list_of_Indexes=list_of_val_index,batch_size=batch_size,dataSet=validDataSet,labels=labels_val, expand_dim = expand,dim =dim_shape)
  checkpointer = ModelCheckpoint(filepath = weights_name, verbose = 1, save_best_only = False)
  hist = model.fit_generator(epochs = epochs, generator=tra_generator,validation_data=val_generator,use_multiprocessing=True,workers=0)
  model.save_weights(weights_name)
  
  
def generateSleepWakeDataSet(signal_data1,signal_data2):
  stageSleep, sleep_test, sleep_val = getSleepStageTrainingAndValidationData(signal_data1,signal_data2,500,50,107)
  awake,awake_test,awake_val = getStageTrainingAndValidationData(signal_data1,signal_data2,0,500,50,107)
  
  frames = [awake, stageSleep]
  dataSet = pd.concat(frames)

  frames1 = [awake_test, sleep_test]
  testDataSet = pd.concat(frames1)

  frames2 = [awake_val, sleep_val]
  validDataSet = pd.concat(frames2)
  return dataSet, testDataSet, validDataSet
  
def generateDataSets(signal_data1,signal_data2): 
  signal_data2= signal_data2.sample(frac=1)
  signal_data1 = signal_data1.sample(frac=1)
  awake,awake_test,awake_val = getStageTrainingAndValidationData(signal_data1,signal_data2,0,500,50,107) 
  rem, rem_test, rem_val = getStageTrainingAndValidationData(signal_data1,signal_data2,4,500,50,107)
  stageSleep1, sleep_test1, sleep_val1 = getStageTrainingAndValidationData(signal_data1,signal_data2,1,500,50,100)
  stageSleep2, sleep_test2, sleep_val2 = getStageTrainingAndValidationData(signal_data1,signal_data2,2,500,50,100)
  stageSleep3, sleep_test3, sleep_val3 = getStageTrainingAndValidationData(signal_data1,signal_data2,3,500,50,100)

  frames = [awake, rem, stageSleep1, stageSleep2, stageSleep3]
  dataSet = pd.concat(frames)

  frames1 = [awake_test, rem_test,sleep_test1,sleep_test2,sleep_test3]
  testDataSet = pd.concat(frames1)

  frames2 = [awake_val,rem, sleep_val1,sleep_val2,sleep_val3]
  validDataSet = pd.concat(frames2)
  return dataSet, testDataSet, validDataSet

def getFirstLastIndex(sizeStep, len_shape):
  data2,y_labels = prepare_data(data_shape =(1,sizeStep,2))
  y_labels[y_labels>4]=0
  res = next(x for x, val in enumerate(y_labels[:len_shape]) if val > 0) 
  res2 = next(x for x, val in enumerate(reversed(y_labels[:len_shape])) if val > 0) 
  res2 = res+res2
  
  if n_length == 1000:
    res2 = res2-5
  elif n_length == 3000:
    res2 = res2-165
  else:
    res2 = res2-1000
  
  return res-10,res2

In [8]:
signal_data1 = pd.DataFrame(columns=[['Class','Signal']])
signal_data2 = pd.DataFrame(columns=[['Class','Signal']])
    
hypnogram_files = [x for x in os.listdir('/content/gdrive/My Drive/EEG/Test') if x.endswith("gram.edf")]
eeg_files = [x for x in os.listdir('/content/gdrive/My Drive/EEG/Test') if x.endswith("PSG.edf")]

print(hypnogram_files.sort())
print(eeg_files.sort())

for i in range(0,7,1):
    hip_fil ='/content/gdrive/My Drive/EEG/Test/'+hypnogram_files[i]
    eeg_fil ='/content/gdrive/My Drive/EEG/Test/'+eeg_files[i]
    data,sf = retrieveHypnogramData_2v(hip_fil,eeg_fil)
    sig_data1,sig_data2 = getChannelData([eeg_fil], data)
    frames = [signal_data1, sig_data1]
    signal_data1 = pd.concat(frames)
    frames2 = [signal_data2, sig_data2]
    signal_data2 = pd.concat(frames2)
    
    
print(signal_data1.shape)
print(signal_data2.shape)

None
None
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal'

In [0]:
#'Generates data for Keras'
class MyDatGen(keras.utils.Sequence):
    def __init__(self, list_of_Indexes, dataSet, labels, batch_size=64, dim=(64,1000,2), n_channels=1,
                 n_classes=2, shuffle=True,expand_dim=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_of_Indexes = list_of_Indexes
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.dataSet = dataSet
        self.shuffle = shuffle
        self.expand_dim = expand_dim
        self.on_epoch_end()
        
#'Denotes the number of batches per epoch'
    def __len__(self):      
        return int(np.floor(len(self.list_of_Indexes) / self.batch_size))
    # 'Generate one batch of data'
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_of_Indexes[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_of_Indexes))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_indexes_temp):
        # Initialization
        #X = np.empty((self.dim))
        y = np.empty((self.batch_size), dtype=int)
        #s = self.dataSet.iloc[0]['Signal']
        chunk = self.dim[1]
        parts = int(3000/self.dim[1])
        k = 0
        a = np.empty([self.batch_size,self.dim[2],self.dim[1]])
        # Generate data
        for i, ID in enumerate(list_indexes_temp):
            stepOne = 0
            cl = self.dataSet.iloc[ID]['Class']
            sig = self.dataSet.iloc[ID]['Signal']
            si = sig[0]; sign = si[0]; sign2 = si[1]; clas = cl[0]
            for j in range(parts):
                if k!=self.batch_size:
                    a[k,:] = np.array([sign[stepOne:(stepOne+chunk)],sign2[stepOne:(stepOne+chunk)]])
                    stepOne = stepOne+chunk
                    y[k] = clas
                    k = k+1
                else:
                    break
        w = a
        a_new = np.reshape(w,(w.shape[0],w.shape[2],2))
        
        if(self.expand_dim):
          a_new = np.expand_dims(a_new, axis=1)
          
        X = a_new
        return X, to_categorical(y)


In [0]:
def get_lstm_network(n_length, n_signals):
  print('********************************************LSTM Arhitecture***************************************************')
  model = Sequential()
  model.add(LSTM(256,input_shape=(n_length, n_signals), return_sequences=True)) #should be 1000X2 signal shape 
  model.add(LSTM(256))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model 

def get_lstm_cnn2(n_steps,n_length,n_signals):
  print('********************************************CNN(2 layer) + LSTM Arhitecture***************************************************')

  model = Sequential()

  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
                            input_shape=(n_steps,n_length,n_signals)))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Dropout(0.5)))
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(100))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras_auc])
  return model

def get_cnn_8layer(n_length,n_signals):
  print('********************************************CNN Arhitecture***************************************************')
  model = Sequential()

  model.add(Convolution1D(filters=16, kernel_size=3, activation='relu',input_shape=(n_length,n_signals)))
  model.add(BatchNormalization())
  model.add(Convolution1D(filters=16, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(SpatialDropout1D(rate = 0.01))

  model.add(Convolution1D(filters=32, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Convolution1D(filters=32, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(SpatialDropout1D(rate = 0.01))

  model.add(Convolution1D(filters=32, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Convolution1D(filters=32, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(SpatialDropout1D(rate = 0.01))

  model.add(Convolution1D(filters=256, kernel_size=3, activation='relu'))
  model.add(BatchNormalization()) 
  model.add(Convolution1D(filters=256, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))

  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  #model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[keras_auc])
  model.compile(loss='logcosh', optimizer=adam, metrics=[keras_auc])
  return model

def get_cnn_lstm_8layer(n_steps,n_length,n_signals): 
  print('********************************************CNN(8 layer) + LSTM Arhitecture***************************************************')
  model = Sequential()

  model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),
                            input_shape=(n_steps,n_length,n_signals)))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(SpatialDropout1D(rate = 0.01)))

  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(SpatialDropout1D(rate = 0.01)))

  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(SpatialDropout1D(rate = 0.01)))

  model.add(TimeDistributed(Conv1D(filters=256, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=256, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))

  model.add(LSTM(100))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras_auc])
  return model

def get_2Class_cnn_lstm_8layer(n_steps,n_length,n_signals): 
  print('********************************************CNN(8 layer) + LSTM Arhitecture***************************************************')
  model = Sequential()

  model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),
                            input_shape=(n_steps,n_length,n_signals)))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(SpatialDropout1D(rate = 0.01)))

  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(SpatialDropout1D(rate = 0.01)))

  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(SpatialDropout1D(rate = 0.01)))

  model.add(TimeDistributed(Conv1D(filters=256, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(Conv1D(filters=256, kernel_size=3, activation='relu')))
  model.add(BatchNormalization())
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))

  model.add(LSTM(100))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(2, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras_auc])
  return model

In [0]:
dataSet, testDataSet, validDataSet = generateDataSets(signal_data1,signal_data2);#5 classes

In [0]:

table = BeautifulTable()
table.column_headers = ["Arhitecture","epohs","acc","confMat"]
                        #"MSE", "Correlation Coeficients","Accuracy","Sleep phase correlation","F1-Score","Accuracy-Sleep", "confussionMat"]

In [0]:
n_outputs = 5 #Binary Classificatio
n_steps, n_length, n_signals,n_batch = 1,500, 2, 64
verbose, epochs, batch_size = True, 30, 128
clases = 5
weight = False
for j in range(10,11,1):
#for j in range(3,4,1):
  epochs = 10*j
  if n_length == 1000:
    interval = 23800
  elif n_length == 3000:
    interval = 2650
  else:
    interval = -1
  startIndex,endIndex = getFirstLastIndex(n_length,interval)


  model = get_cnn_lstm_8layer(n_steps,n_length,n_signals)
  weights_name = '/content/gdrive/My Drive/EEG/CNN8LSTM'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  model.load_weights(weights_name)
  #train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  filename = 'CNN-8 - LSTM Predicted Hypnogram'
  y_true, y_pred = predict_eeg_signal(model,filename,clases,dat_shape = (n_steps,n_length,n_signals))
  table.append_row(["CNN(8layer)+LSTM", epochs,
 #                   mean_squared_error(y_true[0:interval], y_pred[0:interval]),
 #                   np.corrcoef(y_true[0:interval], y_pred[0:interval]),
 #                   accuracy_score(y_true[0:interval], y_pred[0:interval]),
 #                   np.corrcoef(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
 #                   f1_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval],average=None),
                    accuracy_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
                    confusion_matrix(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval])
                   ])
  
  #BatchNormalization
  model = get_lstm_cnn2(n_steps, n_length, n_signals)
  weights_name = '/content/gdrive/My Drive/EEG/CNN2_LSTM_'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  filename = 'sample_test_hypnogram_LSTM_CNN'+str(epochs)
  y_true, y_pred = predict_eeg_signal(model,filename,clases,dat_shape = (n_steps,n_length,n_signals))
  #model.load_weights(weights_name)
  table.append_row(["CNN(2layer)+LSTM", epochs, 
  #                  mean_squared_error(y_true[0:interval], y_pred[0:interval]),
  #                  np.corrcoef(y_true[0:interval], y_pred[0:interval]),
  #                  accuracy_score(y_true[0:interval], y_pred[0:interval]),
  #                  np.corrcoef(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
  #                  f1_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval],average=None),
                    accuracy_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
                    confusion_matrix(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval])
                   ])
 
  model = get_cnn_8layer(n_length, n_signals)
  weights_name = '/content/gdrive/My Drive/EEG/CNN'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  #rain_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, expand=False, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  model.load_weights(weights_name)
  filename = 'sample_test_hypnogram_CNN'+str(epochs)
  y_true, y_pred = predict_eeg_signal(model,filename,clases,expand_dim=False,dat_shape = (n_steps,n_length,n_signals))
  table.append_row(["CNN",  epochs,
  #                  mean_squared_error(y_true[0:interval], y_pred[0:interval]),
  #                  np.corrcoef(y_true[0:interval], y_pred[0:interval]),
  #                  accuracy_score(y_true[0:interval], y_pred[0:interval]),
  #                  np.corrcoef(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
  #                  f1_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval],average=None),
                    accuracy_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
                    confusion_matrix(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval])
                   ])
  
  model = get_lstm_network(n_length, n_signals)
  #model.summary()
  weights_name = '/content/gdrive/My Drive/EEG/LSTM'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  model.load_weights(weights_name)
  #train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, expand=False, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  filename = 'sample_test_hypnogram_LSTM'+str(epochs)
  y_true, y_pred = predict_eeg_signal(model,filename,clases,expand_dim=False,dat_shape = (n_steps,n_length,n_signals))
  table.append_row(["LSTM", epochs,
  #                  mean_squared_error(y_true[0:interval], y_pred[0:interval]),
  #                  np.corrcoef(y_true[0:interval], y_pred[0:interval]),
  #                  accuracy_score(y_true[0:interval], y_pred[0:interval]),
  #                  np.corrcoef(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
  #                  f1_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval],average=None),
                    accuracy_score(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval]),
                    confusion_matrix(y_true[startIndex:endIndex-interval], y_pred[startIndex:endIndex-interval])
                   ])
  
filename = 'Original_hypnogram'
plot_original_eeg(filename,dat_shape = (n_steps,n_length,n_signals))


In [0]:
print(table)

In [0]:
dataSet, testDataSet, validDataSet = generateSleepWakeDataSet(signal_data1,signal_data2);#2 classes

In [0]:
table = BeautifulTable()
table.column_headers = ["Arhitecture","epohs","confMat"]
                        #"MSE", "Correlation Coeficients","Accuracy","Sleep phase correlation","F1-Score","Accuracy-Sleep", "confussionMat"]

In [18]:
n_outputs = 2 #Binary Classificatio
n_steps, n_length, n_signals,n_batch = 1,500, 2, 64
verbose, epochs, batch_size = True, 30, 128
clases = 2
weight = False
#for j in range(10,11,1):
for j in range(5,6,1):
  epochs = 10*j
  if n_length == 1000:
    interval = 7500
  elif n_length == 3000:
    interval = 3650
  else:
    interval = -1
  startIndex,endIndex = getFirstLastIndex(n_length,interval)  
  
  
  
  model = get_lstm_cnn2(n_steps, n_length, n_signals)
  weights_name = '/content/gdrive/My Drive/EEG/CNN2LSTM_2Class_'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  Fmodel.load_weights(weights_name)
  #train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  filename = 'sample_test_hypnogram_LSTM_CNN'+str(epochs)
  y_true1, y_pred = predict_eeg_signal_sleep(model,filename,clases,dat_shape = (n_steps,n_length,n_signals))
  y_true = y = np.empty((len(y_pred)), dtype=int)
  for i in range(len(y_pred)-1):
    if y_true1[i][0] == 1:
      y_true[i] = 0
    if y_true1[i][1] == 1:
      y_true[i] = 1
  table.append_row(["CNN2 - LSTM", epochs,
                      confusion_matrix(y_true[3000:-1000], y_pred[3000:-1000])
                   ])

  model = get_cnn_8layer(n_length, n_signals)
  weights_name = '/content/gdrive/My Drive/EEG/CNN_2class_'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, expand=False, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  #model.load_weights(weights_name)
  filename = 'sample_test_hypnogram_CNN'+str(epochs)
  y_true1, y_pred = predict_eeg_signal_sleep(model,filename,clases,expand_dim=False,dat_shape = (n_steps,n_length,n_signals))
  y_true = y = np.empty((len(y_pred)), dtype=int)
  for i in range(len(y_pred)-1):
    if y_true1[i][0] == 1:
      y_true[i] = 0
    if y_true1[i][1] == 1:
      y_true[i] = 1
  table.append_row(["CNN", epochs,
                       confusion_matrix(y_true[3000:-1000], y_pred[3000:-1000])
                   ])
   
  
  
  model = get_2Class_cnn_lstm_8layer(n_steps,n_length,n_signals)
  #model.summary()
  weights_name = '/content/gdrive/My Drive/EEG/CNN8_LSTM_2Class'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  #model.load_weights(weights_name)
  train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, expand=True, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  filename = 'sample_test_hypnogram_LSTM_CNN'+str(epochs) 
  y_true1, y_pred = predict_eeg_signal_sleep(model,filename,2,expand_dim=True,dat_shape = (n_steps,n_length,n_signals))

 
  y_true = y = np.empty((len(y_pred)), dtype=int)
  for i in range(len(y_pred)-1):
    if y_true1[i][0] == 1:
      y_true[i] = 0
    if y_true1[i][1] == 1:
      y_true[i] = 1
  #print(y_true)
  #print(y_pred)
  table.append_row(["CNN-8 - LSTM", epochs,
                        confusion_matrix(y_true[3000:-1000], y_pred[3000:-1000])
                   ])
  
  
  
  model = get_lstm_network(n_length, n_signals)
  weights_name = '/content/gdrive/My Drive/EEG/LSTM_2Class_'+str(epochs)+str(n_length)+'.weights.best.hdf5'
  #model.load_weights(weights_name)
  train_model(model, dataSet,validDataSet,weights_name, epochs, batch_size, weight_loaded = weight, expand=False, verbose = True,dim_shape = (n_batch,n_length,n_signals))
  filename = 'sample_test_hypnogram_LSTM'+str(epochs)
  y_true1, y_pred = predict_eeg_signal_sleep(model,filename,clases,expand_dim=False,dat_shape = (n_steps,n_length,n_signals))
  
  y_true = y = np.empty((len(y_pred)), dtype=int)
  for i in range(len(y_pred)-1):
    if y_true1[i][0] == 1:
      y_true[i] = 0
    if y_true1[i][1] == 1:
      y_true[i] = 1
  table.append_row(["LSTM", epochs,
                      confusion_matrix(y_true[3000:-1000], y_pred[3000:-1000])
                   ])
  
  

['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']
(15894, 1, 500, 2)
********************************************CNN Arhitecture***************************************************
Epoch 1/50
7/7 [==============================] - 6s 788ms/step - loss: 0.1174 - keras_auc: 0.5876 - val_loss: 0.1200 - val_keras_auc: 0.6729
Epoch 2/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0677 - keras_auc: 0.7340 - val_loss: 0.1215 - val_keras_auc: 0.7695
Epoch 3/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0412 - keras_auc: 0.8084 - val_loss: 0.0958 - val_keras_auc: 0.8318
Epoch 4/50
7/7 [==============================] - 1s 182ms/step - loss: 0.0299 - keras_auc: 0.8551 - val_loss: 0.0620 - val_keras_auc: 0.8720
Epoch 5/50
7/7 [==============================] - 1s 183ms/step - loss: 0.0225

In [0]:
print(table)

In [0]:

#data,sf = retrieveHypnogramData(hip_fil,eeg_fil)
#print(data)
#print(hypnogram_files[0])
#filename = 'Original_hypnogram'
#plot_original_eeg(filename,dat_shape = (n_steps,n_length,n_signals))
import numpy as np
ping = [x for x in os.listdir('/content/') if x.endswith(".csv")]
#print(ping)
for i in range(0,len(ping),1):
  myFile = np.genfromtxt(ping[i], delimiter=',')
  #show_sample_hypnogram(ping[i], start=950,stop=1800, title='SC4001EC-Hypnogram.csv')
  show_sample_hypnogram(ping[i], start=startIndex-100,stop=endIndex-interval-500,title ='Predictions for {}'.format(os.path.basename(ping[i])))
  #plot_hypnogram(myFile, title ='Predictions for {}'.format(os.path.basename(ping[i])))
  '/content/gdrive/My Drive/EEG/Test/SC4001EC-Hypnogram.edf'
  '/content/gdrive/My Drive/EEG/Test/SC4001E0-PSG.edf'
#filename = 'Original_hypnogram'
#plot_original_eeg(filename,dat_shape = (n_steps,1000,n_signals))
#print(endIndex)
#print(startIndex)

In [0]:
"""write/Read hypnogram data.

Write hypnogram data
--------------------
-> write_hypno
    -> Export either using time code
        -> .txt, .csv, .xlsx
    -> Export using one sample per second
        -> .txt, .hyp

Read hypnogram data
-------------------
-> read_hypno
    -> detect_hypno_version
        -> Sample :
            -> .txt, .csv, .xlsx
        -> Tieme :
            -> .txt, .hyp
"""
import os
import logging
import numpy as np


__all__ = ('oversample_hypno', 'write_hypno', 'read_hypno')

logger = logging.getLogger('visbrain')


###############################################################################
###############################################################################
#                              HYPNO CONVERSION
###############################################################################
###############################################################################
def is_xlrd_installed(raise_error=False):
    """Test if xlrd is installed."""
    try:
        import xlrd  # noqa
        is_installed = True
    except:
        is_installed = False
    # Raise error (if needed) :
    if raise_error and not is_installed:
        raise IOError("xlrd not installed. In a terminal, run : pip install "
                      "xlrd")
    return is_installed

def is_pandas_installed(raise_error=False):
    """Test if pandas is installed."""
    try:
        import pandas  # noqa
        is_installed = True
    except:
        is_installed = False
    # Raise error (if needed) :
    if raise_error and not is_installed:
        raise IOError("pandas not installed. See https://pandas.pydata.org/#"
                      "best-way-to-install for installation instructions.")
    return is_installed

def vispy_array(data, dtype=np.float32):
    """Check and convert array to be compatible with buffers.

    Parameters
    ----------
    data : array_like
        Array of data.
    dtype : type | np.float32
        Futur type of the array.

    Returns
    -------
    data : array_like
        Contiguous array of type dtype.
    """
    if not data.flags['C_CONTIGUOUS']:
        data = np.ascontiguousarray(data, dtype=dtype)
    if data.dtype != dtype:
        data = data.astype(dtype, copy=False)
    return data

def transient(data, xvec=None):
    """Perform a transient detection on hypnogram.

    Parameters
    ----------
    data : array_like
        The hypnogram data.
    xvec : array_like | None
        The time vector to use. If None, np.arange(len(data)) will be used
        instead.

    Returns
    -------
    t : array_like
        Hypnogram's transients.
    st : array_like
        Either the transient index (as type int) if xvec is None, or the
        converted version if xvec is not None.
    stages : array_like
        The stages for each segment.
    """
    # Transient detection :
    t = list(np.nonzero(np.abs(data[:-1] - data[1:]))[0])
    # Add first and last points :
    idx = np.vstack((np.array([-1] + t) + 1, np.array(t + [len(data) - 1]))).T
    # Get stages :
    stages = data[idx[:, 0]]
    # Convert (if needed) :
    if (xvec is not None) and (len(xvec) == len(data)):
        st = idx.copy().astype(float)
        st[:, 0] = xvec[idx[:, 0]]
        st[:, 1] = xvec[idx[:, 1]]
    else:
        st = idx

    return np.array(t), st, stages.astype(int)
  
  

def hypno_time_to_sample(df, npts):
    """Convert the hypnogram from a defined timings to a number of samples.

    Parameters
    ----------
    df : pandas.DataFrame
        The data frame that contains timing.
    npts : int, array_like
        Number of time points in the final hypnogram. Alternatively, if npts is
        an array it will be interprated as the time vector.

    Returns
    -------
    hypno : array_like
        Hypnogram data of shape (npts,).
    time : array_like
        Time vector of shape (npts,).
    sf_hyp : float
        Sampling frequency of the hypnogram.
    """
    # Drop lines that contains * :
    drop_rows = np.char.find(np.array(df['Stage']).astype(str), '*')
    df = df.iloc[drop_rows.astype(bool)]
    df.is_copy = False  # avoid pandas warning
    # Replace text by numerical values :
    to_replace = ['Wake', 'N1', 'N2', 'N3', 'REM', 'Art']
    values = [0, 1, 2, 3, 4, -1]
    df.replace(to_replace, values, inplace=True)
    # Get stages and time index :
    stages = np.array(df['Stage']).astype(str)
    time_idx = np.array(df['Time']).astype(float)
    # Compute time vector and sampling frequency :
    if isinstance(npts, np.ndarray):
        time = npts.copy()
    elif isinstance(npts, int):
        time = np.arange(npts) * time_idx[-1] / (npts - 1)
    sf_hyp = 1. / (time[1] - time[0])
    # Find closest time index :
    index = np.abs(time.reshape(-1, 1) - time_idx.reshape(1, -1))
    index = np.r_[0, index.argmin(0) + 1]
    # Fill the hypnogram :
    hypno = np.zeros((len(time),), dtype=int)
    for k in range(len(index) - 1):
        hypno[index[k]:index[k + 1]] = int(stages[k])
    return hypno, time, sf_hyp


def hypno_sample_to_time(hypno, time):
    """Convert the hypnogram from a number of samples to a defined timings.

    Parameters
    ----------
    hypno : array_like
        Hypnogram data.
    time : array_like
        The time vector.

    Returns
    -------
    df : pandas.DataFrame
        The data frame that contains all of the transient timings.
    """
    # Test if panda is installed :
    is_pandas_installed(True)
    import pandas as pd
    # Transient detection :
    _, tr, stages = transient(hypno, time)
    # Save the hypnogram :
    items = np.array(['Wake', 'N1', 'N2', 'N3', 'REM', 'Art'])
    return pd.DataFrame({'Stage': items[stages], 'Time': tr[:, 1]})


def oversample_hypno(hypno, n):
    """Oversample hypnogram.

    Parameters
    ----------
    hypno : array_like
        Hypnogram data of shape (N,) with N < n.
    n : int
        The destination length.

    Returns
    -------
    hypno : array_like
        The hypnogram of shape (n,)
    """
    # Get the repetition number :
    rep_nb = int(np.round(n / len(hypno)))

    # Repeat hypnogram :
    hypno = np.repeat(hypno, rep_nb)
    npts = len(hypno)

    # Check size
    if npts < n:
        hypno = np.append(hypno, hypno[-1] * np.ones((n - npts)))
    elif npts > n:
        hypno = hypno[0:n]

    return hypno.astype(int)


###############################################################################
###############################################################################
#                               WRITE HYPNO
###############################################################################
###############################################################################

def write_hypno(filename, hypno, version='time', sf=100., npts=1, window=1.,
                time=None, info=None):
    """Save hypnogram data.

    Parameters
    ----------
    filename : str
        Filename (with full path) of the file to save
    hypno : array_like
        Hypnogram array, same length as data
    sf : float | 100.
        Original sampling rate of the raw data
    npts : int | 1
        Original number of points in the raw data
    window : float | 1
        Time window (second) of each point in the hypno
        Default is one value per second
        (e.g. window = 30 = 1 value per 30 second)
    time : array_like | None
        The time vector.
    info : dict | None
        Additional informations to add to the file (prepend with *).
    """
    # Checking :
    assert isinstance(filename, str)
    assert isinstance(hypno, np.ndarray)
    assert version in ['time', 'sample']
    # Extract file extension :
    _, ext = os.path.splitext(filename)
    # Switch between time and sample version :
    if version is 'sample':  # v1 = sample
        # Take a down-sample version of the hypno :
        step = int(len(hypno) / np.round(npts / sf))
        hypno = hypno[::step].astype(int)
        # Export :
        if ext == '.txt':
            _write_hypno_txt_sample(filename, hypno, window=window)
        elif ext == '.hyp':
            _write_hypno_hyp_sample(filename, hypno, sf=sf, npts=npts)
    elif version is 'time':  # v2 = time
        # Get the DataFrame :
        df = hypno_sample_to_time(hypno, time)
        if isinstance(info, dict):
            is_pandas_installed(True)
            import pandas as pd
            info = {'*' + k: i for k, i in info.items()}
            df_info = pd.DataFrame({'Stage': list(info.keys()),
                                    'Time': list(info.values())})
            df = df_info.append(df)
        if ext in ['.txt', '.csv']:
            df.to_csv(filename, header=None, index=None, sep='\t', mode='a')
        elif ext == '.xlsx':
            is_pandas_installed(True)
            is_xlrd_installed(True)
            import pandas as pd
            writer = pd.ExcelWriter(filename)
            df.to_excel(writer, sheet_name='Data', index=False, header=False)
            writer.save()
    logger.info("Hypnogram saved (%s)" % filename)


def _write_hypno_txt_sample(filename, hypno, window=1.):
    """Save hypnogram in txt file format (txt).

    Header is in file filename_description.txt

    Parameters
    ----------
    filename : str
        Filename (with full path) of the file to save
    hypno : array_like
        Hypnogram array, same length as data
    window : float | 1
        Time window (second) of each point in the hypno
        Default is one value per second
        (e.g. window = 30 = 1 value per 30 second)
    """
    base = os.path.basename(filename)
    dirname = os.path.dirname(filename)
    descript = os.path.join(dirname, os.path.splitext(
        base)[0] + '_description.txt')

    # Save hypno
    np.savetxt(filename, hypno, fmt='%s')

    # Save header file
    hdr = np.array([['time ' + str(window)], ['W 0'], ['N1 1'], ['N2 2'],
                    ['N3 3'], ['REM 4'], ['Art -1']]).flatten()
    np.savetxt(descript, hdr, fmt='%s')


def _write_hypno_hyp_sample(filename, hypno, sf=100., npts=1):
    """Save hypnogram in Elan file format (hyp).

    Parameters
    ----------
    filename : str
        Filename (with full path) of the file to save
    hypno : array_like
        Hypnogram array, same length as data
    sf : float | 100.
        Original sampling rate of the raw data
    npts : int | 1
        Original number of points in the raw data
    """
    hypno[hypno == 4] = 5

    hdr = np.array([['time_base 1.000000'],
                    ['sampling_period ' + str(np.round(1 / sf, 8))],
                    ['epoch_nb ' + str(int(npts / sf))],
                    ['epoch_list']]).flatten()

    # Save
    export = np.append(hdr, hypno.astype(str))
    np.savetxt(filename, export, fmt='%s')


###############################################################################
###############################################################################
#                                 READ HYPNO
###############################################################################
###############################################################################


def read_hypno(filename, time=None, datafile=None):
    """Load hypnogram file.

    Sleep stages in the hypnogram should be scored as follow
    see Iber et al. 2007

    Wake:   0
    N1:     1
    N2:     2
    N3:     3
    REM:    4
    Art:    -1  (optional)

    Parameters
    ----------
    filename : string
        Filename (with full path) to hypnogram file.
    time : array_like | None
        The time vector (used to interpolate Excel files).
    datafile : string | None
        Filename (with full path) to the data file.

    Returns
    -------
    hypno : array_like
        The hypnogram vector in its original length.
    sf_hyp: float
        The hypnogram original sampling frequency (Hz)
    """
    # Test if file exist :
    assert os.path.isfile(filename)

    # Extract file extension :
    file, ext = os.path.splitext(filename)

    # Load the hypnogram :
    if ext == '.hyp':  # v1 = ELAN
        hypno, sf_hyp = _read_hypno_hyp_sample(filename)
    elif ext == '.edf':  # v1 = EDF+
        hypno, sf_hyp = _read_hypno_edf_sample(filename, datafile)
    elif ext in ['.txt', '.csv']:  # [v1, v2] = TXT / CSV
        header = os.path.splitext(filename)[0] + '_description.txt'
        if os.path.isfile(header):  # if there's a header -> v1
            hypno, sf_hyp = _read_hypno_txt_sample(filename)
        else:  # v2
            import pandas as pd
            df = pd.read_csv(filename, delim_whitespace=True, header=None,
                             names=['Stage', 'Time'])
            hypno, _, sf_hyp = hypno_time_to_sample(df, len(time))
    elif ext == '.xlsx':  # v2 = Excel
        import pandas as pd
        df = pd.read_excel(filename, header=None, names=['Stage', 'Time'])
        hypno, _, sf_hyp = hypno_time_to_sample(df, len(time))

    logger.info("Hypnogram successfully loaded (%s)" % filename)

    return vispy_array(hypno), sf_hyp


def _read_hypno_hyp_sample(path):
    """Read Elan hypnogram (hyp).

    Parameters
    ----------
    path : str
        Filename(with full path) to Elan .hyp file

    Returns
    -------
    hypno : array_like
        The hypnogram vector in its original length.
    sf_hyp : float
        The hypnogram original sampling frequency (Hz)
    """
    hyp = np.genfromtxt(path, delimiter='\n', usecols=[0],
                        dtype=None, skip_header=0, encoding='utf-8')

    # Get sampling frequency of hypnogram
    sf_hyp = 1 / float(hyp[0].split()[1])

    # Extract hypnogram values
    hypno = np.array(hyp[4:], dtype=np.int)

    # Replace values according to Iber et al 2007
    hypno[hypno == -2] = -1
    hypno[hypno == 4] = 3
    hypno[hypno == 5] = 4

    return hypno, sf_hyp


def _read_hypno_txt_sample(path):
    """Read text files (.txt / .csv) hypnogram.

    Parameters
    ----------
    path : str
        Filename(with full path) to hypnogram (.txt)

    Returns
    -------
    hypno : array_like
        The hypnogram vector in its original length.
    sf_hyp : float
        The hypnogram original sampling frequency (Hz)
    """
    assert os.path.isfile(path)

    file, ext = os.path.splitext(path)

    header = file + '_description.txt'
    assert os.path.isfile(header)

    # Load header file
    labels = np.genfromtxt(header, dtype=str, delimiter=" ", usecols=0,
                           encoding='utf-8')
    values = np.genfromtxt(header, dtype=float, delimiter=" ", usecols=1,
                           encoding='utf-8')
    desc = {label: row for label, row in zip(labels, values)}

    # Get sampling frequency of hypnogram
    sf_hyp = 1. / float(desc['time'])

    # Load hypnogram file
    hyp = np.genfromtxt(path, delimiter='\n', usecols=[0],
                        dtype=None, skip_header=0, encoding='utf-8')

    if not np.issubdtype(hyp.dtype, np.integer):
        hypno = np.array([s for s in hyp if s.lstrip('-').isdigit()],
                         dtype=int)
    else:
        hypno = hyp.astype(int)

    hypno = swap_hyp_values(hypno, desc)

    return hypno, sf_hyp


def _read_hypno_edf_sample(hypno_file_path, data_file_path):
    """Read hypnogram files which are formatted according to EDF+.

    See file specifications (see https://www.edfplus.info/specs/index.html).
    The function was specifically developed to read hypnograms which are part
    of the Physionet Sleep Database (https://physionet.org/pn4/sleep-edfx/).
    The Physionet Sleep Database contains 61 polysomnograms (PSGs) with
    accompanying hypnograms. The Physionet polysomnogram files are using edf
    format, while the hypnograms use EDF+. When using EDF+ for hypnograms, the
    data records contain Timestamped Annotation Lists (TALs). Each TAL consists
    of an onset, a duration, and a sleep stage. The following assumptions have
    been made:
    - the scoring period for the hypnogram is equal to the number of seconds
      per data record as given by header information of the accompanying
      polysomnogram file
    - all EDF+ hypnogram files use the same scoring for sleep stages which are
      in the function converted to the values used by Visbrain:
            EDF+ score                    Visbrain
          Sleep stage ?                      -1
          Movement time                      -1
          Sleep stage W                       0
          Sleep stage 1                       1
          Sleep stage 2                       2
          Sleep stage 3                       3
          Sleep stage 4                       3
          Sleep stage R                       4

    The Physionet hypnogram files (EDF+) sometimes cover a slightly longer
    total period than the polysomnograph files, where the score for the last
    part is set to "Sleep stage ?". This function will only read the hypnogram
    files so that only the time period of the polysomnograph is covered.

    The function will check whether the hypnogram file is the appropriate
    accompanying file to the polysomnogranph file by comparing the "subjcet_id"
    as well as the "recording_id" of the two files.

    Parameters
    ----------
    data_file_path : str
        Filename(with full path) to data(.edf)
    hypno_file_path : str
        Filename(with full path) to corresponding hypnogram(.edf)

    Returns
    -------
    hypno : array_like
        The hypnogram vector in its original length.
    sf_hyp : float
        The hypnogram original sampling frequency (Hz)
    """
    if not isinstance(data_file_path, str):
        raise IOError("Reading EDF+ need the path to the data file.")
    data_file_path, _ = os.path.splitext(data_file_path)

    with open(data_file_path + '.edf', 'rb') as f:  # open edf data file
        hdr1 = {}
        assert f.tell() == 0
        assert f.read(8) == b'0       '

        # Recording info (patient info and date and time)
        hdr1['subject_id'] = f.read(80).decode('utf-8').strip()
        hdr1['recording_id'] = f.read(80).decode('utf-8').strip()

        f.seek(68, 1)
        hdr1['n_records'] = int(f.read(8))
        hdr1['record_length'] = float(f.read(8))  # in seconds
        end_file = str(int(hdr1['n_records'] * hdr1['record_length']))

    with open(hypno_file_path, 'rb') as f:  # open edf hypnogram file
        hdr2 = {}
        assert f.tell() == 0
        assert f.read(8) == b'0       '

        # Recording info (patient info and date and time)
        hdr2['subject_id'] = f.read(80).decode('utf-8').strip()
        hdr2['recording_id'] = f.read(80).decode('utf-8').strip()

        # Compare the patient info and recording date of the two files
        if not (hdr1['subject_id'] == hdr2['subject_id'] and hdr1[
                'recording_id'] == hdr2['recording_id']):
            raise IOError("Hypnogram file does not match polysomnograph file")

        # skip records not required
        f.seek(16, 1)
        # read bytes in header of hypnogram file
        hdr2['header_n_bytes'] = int(f.read(8))

        # go to the end of the header info```
        f.seek(hdr2['header_n_bytes'])

        data_hypno = f.read().decode('utf-8')  # read the data

    # number of secs per data record used for score period
    time = hdr1['record_length']
    data_hypno_spl = data_hypno.split('\x00')
    ln = len(data_hypno_spl)
    tr = {'Sleep stage ?': -1, 'Movement time': -1, 'Sleep stage W': 0,
          'Sleep stage 1': 1, 'Sleep stage 2': 2, 'Sleep stage 3': 3,
          'Sleep stage 4': 3, 'Sleep stage R': 4}
    hypno_s = []
    for i in range(ln):
        in_start = data_hypno_spl[i].find('\x15')
        if in_start == -1:
            continue
        else:
            in_stop = data_hypno_spl[i].find('\x14')
            onset = data_hypno_spl[i][1:in_start]
            duration = data_hypno_spl[i][in_start + 1:in_stop]

            if int(onset) + int(duration) >= int(end_file):
                duration = str(int(end_file) - int(onset))

            sleepstage = data_hypno_spl[i][in_stop + 1:-1]

            nr = int(int(duration) / 30)
            entry = [tr[sleepstage]] * nr
            hypno_s.extend(entry)

    hypno_s = np.array(hypno_s).astype(int)
    sf_hyp = 1. / time
    return hypno_s, sf_hyp


def swap_hyp_values(hypno, desc):
    """Swap values in hypnogram vector.

    Sleep stages in the hypnogram should be scored as follow
    see Iber et al. 2007

    e.g from the DREAM bank EDF database
    Stage   Orig. val    New val
    W       5           0
    N1      3           1
    N2      2           2
    N3      1           3
    REM     0           4

    Parameters
    ----------
    hypno : array_like
        The hypnogram vector
    description : str
        Path to a .txt file containing labels and values of each sleep
        stage separated by a space

    Returns
    -------
    hypno_s : array_like
        Hypnogram with swapped values
    """
    # Swap values
    hypno_s = -1 * np.ones(shape=(hypno.shape), dtype=int)

    if 'Art' in desc:
        hypno_s[hypno == desc['Art']] = -1
    if 'Nde' in desc:
        hypno_s[hypno == desc['Nde']] = -1
    if 'Mt' in desc:
        hypno_s[hypno == desc['Mt']] = -1
    if 'W' in desc:
        hypno_s[hypno == desc['W']] = 0
    if 'N1' in desc:
        hypno_s[hypno == desc['N1']] = 1
    if 'N2' in desc:
        hypno_s[hypno == desc['N2']] = 2
    if 'N3' in desc:
        hypno_s[hypno == desc['N3']] = 3
    if 'N4' in desc:
        hypno_s[hypno == desc['N4']] = 3
    if 'REM' in desc:
        hypno_s[hypno == desc['REM']] = 4

    return hypno_s


In [0]:
"""Export graphical components as figures.

- write_fig_hyp : Export the hypnogram as a figure
- write_fig_canvas : Export a canvas as a figure
- write_fig_pyqt : Export a GUI window as a figure
"""
import os
import logging
import numpy as np

logger = logging.getLogger('visbrain')

__all__ = ('write_fig_hyp', 'write_fig_spindles', 'write_fig_canvas',
           'write_fig_pyqt')


def write_fig_hyp(data, sf, file=None, start_s=0, grid=False, ascolor=False,
                  dpi=300, colors={-1: '#8bbf56', 0: '#56bf8b', 1: '#aabcce',
                                   2: '#405c79', 3: '#0b1c2c', 4: '#bf5656'}):
    """Export hypnogram to a high-res png figure.

    Parameters
    ----------
    data : array_like
        Hypnogram vector
    sf : float
        The sampling frequency of displayed elements (could be the
        down-sampling frequency)
    file : string | None
        Output filename (with full path). If None, the plot is displayed.
    start_s : float | 0.
        Record starting time given in seconds.
    grid : bool | False
        Plot X and Y grid.
    ascolor : bool | False
        Plot in color
    dpi : int | 600
        Dots per inches
    color : dict | {}
        Color for each sleep stage. Default is : {-1: '#8bbf56', 0: '#56bf8b',
        1: '#aabcce', 2: '#405c79', 3: '#0b1c2c', 4: '#bf5656'}
    """
    import matplotlib.pyplot as plt
    import datetime

    # Internal copy :
    hypno = data.copy()

    # Downsample to get one value per second
    sf = int(sf)
    hypno = hypno[::sf]

    # Put REM between Wake and N1 sleep
    hypno[hypno >= 1] += 1
    hypno[hypno == 5] = 1
    idx_rem = np.where(hypno == 1)[0]
    val_rem = np.zeros(hypno.size)
    val_rem[:] = np.nan
    val_rem[idx_rem] = 1

    # Find if artefacts are present in hypno
    art = True if -1 in hypno else False

    # Start plotting
    fig, ax = plt.subplots(figsize=(8, 3), edgecolor='k')
    lhyp = len(hypno) / 60
    lw = 1.5
    if lhyp < 60:
        xticks = np.arange(0, len(hypno), 10 * 60)
        lw = 2
    elif lhyp < 180 and lhyp > 60:
        xticks = np.arange(0, len(hypno), 30 * 60)
    else:
        xticks = np.arange(0, len(hypno), 60 * 60)

    xticks = np.append(xticks, len(hypno))
    xlabels = (xticks + start_s).astype(int)
    xlabels_str = [str(datetime.timedelta(seconds=int(j)))[:-3]
                   for i, j in enumerate(xlabels)]
    xlabels_str = [s.replace('1 day, ', '') for s in xlabels_str]
    plt.xlim(0, len(hypno))
    plt.xticks(xticks, xlabels_str)
    if not ascolor:
        plt.plot(hypno, 'k', ls='steps', linewidth=lw)
    else:
        for k, i in colors.items():
            # Quick and dirty switch :
            if k == 1:
                q = 2
            elif k == 4:
                q = 1
            elif k in [2, 3]:
                q = k + 1
            else:
                q = k
            mask = np.ones((len(hypno),), dtype=bool)
            idxm = np.where(hypno == q)[0] + 1
            idxm[idxm >= len(hypno)] = len(hypno) - 1
            mask[idxm] = False
            plt.plot(np.ma.masked_array(hypno, mask=mask), i, ls='steps',
                     linewidth=lw)

    # Plot REM epochs
    remcol = 'k' if not ascolor else colors[4]
    for i in np.arange(0.6, 1, 0.01):
        plt.plot(np.arange(len(hypno)), i * val_rem, remcol, linewidth=lw)

    # Y-Ticks and Labels
    if art:
        ylabels = ['Art', 'Wake', 'REM', 'N1', 'N2', 'N3']
        plt.yticks([-1, 0, 1, 2, 3, 4], ylabels)
        plt.ylim(-1.5, 4.5)
    else:
        ylabels = ['', 'Wake', 'REM', 'N1', 'N2', 'N3']
        plt.yticks([-0.5, 0, 1, 2, 3, 4], ylabels)
        plt.ylim(-.5, 4.5)

    # X-Ticks and Labels
    plt.xlabel("Time")
    plt.ylabel("Sleep Stage")

    # Grid
    if grid:
        plt.grid(True, 'major', ls=':', lw=.2, c='k', alpha=.3)

    plt.tick_params(axis='both', which='both', bottom=True, top=False,
                    labelbottom=True, left=True, right=False, labelleft=True,
                    labelcolor='k', direction='out')

    # Invert Y axis and despine
    ax.invert_yaxis()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)

    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.spines['bottom'].set_smart_bounds(True)

    # Save as 600 dpi .png
    if isinstance(file, str):
        plt.savefig(file, format='png', dpi=dpi, bbox_inches='tight')
        logger.info('Image successfully saved (%s)' % file)
        plt.close()
    else:
        plt.show()


def write_fig_spindles(data, sf, hypno=None, file=None, start_s=0.,
                       window_s=10., thr=3., nrem_only=False, dpi=300):
    """Show steps of the spindles detection for a specific time window.

    Parameters
    ----------
    data : array_like
        Data vector
    sf : float
        The sampling frequency of displayed elements (could be the
        down-sampling frequency)
    hypno : array_like | None
        Hypnogram vector
    file : string | None
        Output filename (with full path). If None, the plot is displayed.
    start_s : float | 0.
        Starting point in sec of the window to plot
    window_s : float | 10.
        Window duration in seconds
    thresh : float | 3
        Hard threshold for the spindles detection
    dpi : int | 300
        Dots per inches
    """
    import matplotlib.pyplot as plt
    from ..utils.sleep import spindlesdetect
    from ..utils.filtering import filt

    # Define an empty hypno hypnogram if None :
    hypno = np.zeros_like(data) if hypno is None else hypno

    # Run spindles detection on the selected channel
    (idx_spindles, _, _, dur, pwr, idx_start, idx_stop, hard_thr, soft_thr,
     idx_sigma, fmin, fmax, sigma_nfpow, amplitude,
     sigma_thr) = spindlesdetect(data, sf, thr, hypno, nrem_only,
                                 return_full=True)

    # Define plotting range
    start_sf = int(start_s * sf)
    x = range(start_sf, start_sf + int(window_s * sf))

    # Bandpass filter of the window
    elec_filt = filt(sf, [12., 14.], data[x], order=4)

    # Find beginning and end of spindle within the window
    idx_start_win = idx_start[(idx_start >= min(x)) & (idx_start <= max(x))]
    idx_stop_win = idx_stop[(idx_stop >= min(x)) & (idx_stop <= max(x))]
    sp_in_win = np.in1d(idx_start, idx_start_win)
    sp_power = pwr[sp_in_win]
    sp_duration = dur[sp_in_win]

    # Find indices of spindles within the window
    idx_spindles_win = idx_spindles[
        (idx_spindles >= min(x)) & (idx_spindles <= max(x))]

    # Find indices of sigma power > supra-threshold within window
    idx_sigma_win = idx_sigma[(idx_sigma > min(x)) & (idx_sigma < max(x))]
    # Find indices of wavelet amplitude > supra-threshold within window
    with np.errstate(divide='ignore', invalid='ignore'):
        idx_hard = np.where(amplitude > hard_thr)[0]

    idx_hard_win = idx_hard[(idx_hard > min(x)) & (idx_hard < max(x))]

    # Initialize Y vector
    y_sigma, y_spindles, y_wlt, y_hard = np.empty(len(x)), np.empty(len(x)), \
        np.empty(len(x)), np.empty(len(x))
    y_sigma[:], y_spindles[:], y_wlt[:], y_hard[:] = np.nan, np.nan, np.nan, \
        np.nan

    # Fill Y vector
    y_sigma[idx_sigma_win - start_sf] = sigma_nfpow[idx_sigma_win]
    y_spindles[idx_spindles_win - start_sf] = data[idx_spindles_win]
    y_wlt[idx_spindles_win - start_sf] = amplitude[idx_spindles_win]
    y_hard[idx_hard_win - start_sf] = amplitude[idx_hard_win]

    # Start plot
    f, axarr = plt.subplots(4, figsize=(10, 6), sharex=True)
    f.subplots_adjust(hspace=0.6)

    # Plot original signal
    axarr[0].plot(x, data[x], 'darkslategrey', lw=1.5)
    axarr[0].plot(x, y_spindles, 'brown', lw=1.5)
    axarr[0].set_title('Original signal (' + str(window_s) + ' sec)')
    axarr[0].set_xlim([min(x), max(x)])

    if sp_power.size >= 1:
        text = 'power = ' + str(np.round(sp_power, 2)) + \
            ' - duration = ' + str(sp_duration) + ' ms'
        axarr[0].annotate(text, xy=(min(x), min(data[x])), fontsize=9,
                          xycoords='data')

    # Plot filtered signal
    axarr[1].plot(x, elec_filt, 'darkslategrey', linewidth=1.5)
    axarr[1].set_title('Filtered')

    # Plot wavelet envelope
    axarr[2].plot(x, amplitude[x], 'darkslategrey', linewidth=2)
    axarr[2].plot(x, y_wlt, 'coral', lw=3)
    # axarr[2].plot(x, y_hard, 'indianred', lw=3)
    axarr[2].scatter(idx_start_win, amplitude[idx_start_win], 60, 'coral')
    axarr[2].scatter(idx_stop_win, amplitude[idx_stop_win], 60, 'coral')
    axarr[2].set_title('Wavelet')
    axarr[2].axhline(y=hard_thr, color='grey', linestyle=':', lw=1.5)
    axarr[2].axhline(y=soft_thr, color='grey', linestyle=':', lw=1.5)

    # Plot sigma normalized power
    axarr[3].plot(x, sigma_nfpow[x], 'darkslategrey', linewidth=2)
    axarr[3].set_title('Sigma power')
    axarr[3].plot(x, y_sigma, lw=3, color='lightcoral')
    axarr[3].axhline(y=sigma_thr, color='grey', linestyle=':', lw=1.5)

    # Despine
    for ax in range(4):
        axarr[ax].axis('off')

    # Save as .png
    if isinstance(file, str):
        plt.savefig(file, format='png', dpi=dpi, bbox_inches='tight')
        logger.info('Image successfully saved (%s)' % file)
        plt.close()
    else:
        plt.show()


def write_fig_canvas(filename, canvas, widget=None, autocrop=False,
                     region=None, print_size=None, unit='centimeter', dpi=300.,
                     factor=1., bgcolor=None, transparent=False):
    """Export a canvas as a figure.

    Parameters
    ----------
    filename : string
        Name of the figure to export.
    canvas : VisPy canvas
        The vispy canvas to export.
    widget : PyQt widget | None
        The widget parent of the canvas.
    autocrop : bool | False
        Auto-cropping argument to remove useless space.
    region : tuple/list | None
        The region to export (x_start, y_start, width, height).
    print_size : tuple | None
        The desired print size. This argument should be used in association
        with the dpi and unit inputs. print_size describe should be a tuple
        of two floats describing (width, height) of the exported image for
        a specific dpi level. The final image might not have the exact
        desired size but will try instead to find a compromize
        regarding to the proportion of width/height of the original image.
    unit : {'centimeter', 'millimeter', 'pixel', 'inch'}
        Unit of the printed size.
    dpi : float | 300.
        Dots per inch for printing the image.
    factor : float | None
        If you don't want to use the print_size input, factor simply
        multiply the resolution of your screen.
    bgcolor : array_like/string | None
        Background color of the canvas.
    transparent : bool | False
        Use transparent background.
    """
    from ..utils import piccrop
    from vispy.io import imsave

    # Get the size of the canvas and backend :
    c_size = canvas.size
    b_size = canvas._backend._physical_size

    # If the GUI is displayed, c_size and b_size should be equals. If not,
    # and if the canvas is resizable, the canvas might have a different size
    # because it hasn't been updated. In that case, we force the canvas to have
    # the same size as the backend :
    if c_size != b_size:
        canvas.size = b_size

    # Backup size / background color :
    backup_size = canvas.physical_size
    backup_bgcolor = canvas.bgcolor

    # dpi checking :
    if print_size is None:
        logger.warning("dpi parameter is not active if `print_size` is None. "
                       "Use for example `print_size=(5, 5)`")

    # User select a desired print size with at a specific dpi :
    if print_size is not None:
        # Type checking :
        if not isinstance(print_size, (tuple, list)):
            raise TypeError("The print_size must either be a tuple or a list "
                            "describing the (width, height) of the"
                            " image in %s" % unit)
        # Check print size :
        if not all([isinstance(k, (int, float)) for k in print_size]):
            raise TypeError("print_size must be a tuple describing the "
                            "(width, height) of the image in %s" % unit)

        print_size = np.asarray(print_size)
        # If the user select the auto-croping option, the canvas must be render
        # before :
        if autocrop:
            img = canvas.render()
            s_output = piccrop(img)[:, :, 0].shape
            logger.info("Image cropped to closest non-backround pixels")
        else:
            s_output = b_size
        # Unit conversion :
        if unit == 'millimeter':
            mult = 1. / (10. * 2.54)
        elif unit == 'centimeter':
            mult = 1. / 2.54
        elif unit == 'pixel':
            mult = 1. / dpi
        elif unit == 'inch':
            mult = 1.
        else:
            raise ValueError("The unit must either be 'millimeter', "
                             "'centimeter', 'pixel' or 'inch' and not " + unit)
        # Get the factor to apply to the canvas size. This factor is defined as
        # the mean required float to get either the desired width/height.
        # Note that the min or the max can also be used instead.
        factor = np.mean(print_size * dpi * mult / np.asarray(s_output))

    # Multply the original canvas size :
    if factor is not None:
        # Get the new width and height :
        new_width = int(b_size[0] * factor)
        new_height = int(b_size[1] * factor)
        # Set it to the canvas, backend and the widget :
        canvas._backend._vispy_set_physical_size(new_width, new_height)
        canvas.size = (new_width, new_height)
        if widget is not None:
            widget.size = (new_width, new_height)

    # Background color and transparency :
    if bgcolor is not None:
        canvas.bgcolor = color2vb(bgcolor, alpha=1.)
    if transparent:
        canvas.bgcolor = [0.] * 4

    # Render the canvas :
    try:
        img = canvas.render(region=region)
    except:
        raise ValueError("Can not render the canvas. Try to decrease the "
                         "resolution")

    # Remove alpha for files that are not png or tiff :
    if os.path.splitext(filename)[1] not in ['.png', '.tiff']:
        img = img[..., 0:-1]

    # Apply auto-cropping to the image :
    if autocrop:
        img = piccrop(img)
        logger.info("Image cropped to closest non-backround pixels")
    # Save it :
    imsave(filename, img)
    px = tuple(img[:, :, 0].T.shape)
    logger.info("Image of size %rpx successfully saved (%s)" % (px, filename))

    # Set to the canvas it's previous size :
    canvas._backend._physical_size = backup_size
    canvas.size = backup_size
    canvas.bgcolor = backup_bgcolor


def write_fig_pyqt(self, filename):
    """Export a GUI window as a figure.

    Parameters
    ----------
    self : struct
        The self structure of the GUI window.
    filename : string
        The picture file name.
    """
    from PyQt5 import QtWidgets, QtCore

    # Screnshot function :
    def _take_screenshot():
        """Take the screenshot."""
        screen = QtWidgets.QApplication.primaryScreen()
        p = screen.grabWindow(0)
        p.save(filename)
        logger.info('Image successfully saved (%s)' % filename)
    # Take screenshot if filename :
    if filename:
        # Timer (avoid shooting the saving window)
        self.timerScreen = QtCore.QTimer()
        # self.timerScreen.setInterval(100)
        self.timerScreen.setSingleShot(True)
        self.timerScreen.timeout.connect(_take_screenshot)
        self.timerScreen.start(500)
        
def color2vb(color=None, default=(1., 1., 1.), length=1, alpha=1.0,
             faces_index=False):
    """Turn into a RGBA compatible color format.

    This function can tranform a tuple of RGB, a matplotlib color or an
    hexadecimal color into an array of RGBA colors.

    Parameters
    ----------
    color : None/tuple/string | None
        The color to use. Can either be None, or a tuple (R, G, B),
        a matplotlib color or an hexadecimal color '#...'.
    default : tuple | (1,1,1)
        The default color to use instead.
    length : int | 1
        The length of the output array.
    alpha : float | 1
        The opacity (Last digit of the RGBA tuple).
    faces_index : bool | False
        Specify if the returned color have to be compatible with faces index
        (e.g a (n_color, 3, 4) array).

    Return
    ------
    vcolor : array_like
        Array of RGBA colors of shape (length, 4).
    """
    # Default or static color :
    if (color is None) or isinstance(color, (str, tuple, list, np.ndarray)):
        if color is None:  # Default
            coltuple = default
        elif isinstance(color, (tuple, list, np.ndarray)):  # Static
            color = np.squeeze(color).ravel()
            if len(color) == 4:
                alpha = color[-1]
                color = color[0:-1]
            coltuple = color
        elif isinstance(color, str) and (color[0] is not '#'):  # Matplotlib
            # Check if the name is in the Matplotlib database :
            if color in mplcol.cnames.keys():
                coltuple = mplcol.hex2color(mplcol.cnames[color])
            else:
                warn("The color name " + color + " is not in the matplotlib "
                     "database. Default color will be used instead.")
                coltuple = default
        elif isinstance(color, str) and (color[0] is '#'):  # Hexadecimal
            try:
                coltuple = mplcol.hex2color(color)
            except:
                warn("The hexadecimal color " + color + " is not valid. "
                     "Default color will be used instead.")
                coltuple = default
        # Set the color :
        vcolor = np.concatenate((np.array([list(coltuple)] * length),
                                 alpha * np.ones((length, 1),
                                                 dtype=np.float32)), axis=1)

        # Faces index :
        if faces_index:
            vcolor = np.tile(vcolor[:, np.newaxis, :], (1, 3, 1))

        return vcolor.astype(np.float32)
    else:
        raise ValueError(str(type(color)) + " is not a recognized type of "
                         "color. Use None, tuple or string")
